In [48]:
import numpy as np
import torch
import pandas as pd
import matplotlib as plt
from torch.nn import Linear, ReLU, BatchNorm2d
import torch.nn as nn
# import torch.nn.functional.glu as GLU

In [23]:
dfs = pd.read_csv("adult.data", header=None)

In [24]:
cols = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'earn']
dfs.columns = cols

In [25]:
dfs.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,earn
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [26]:
dfs.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [27]:
print("Workclasses: ", dfs['workclass'].unique())
print("Fnlwgt: ", dfs['fnlwgt'].unique())
print("Education: ", dfs['education'].unique())
print("Marital Status: ", dfs['marital-status'].unique())
print("Occupation: ", dfs['occupation'].unique())
print("Relationship: ", dfs['relationship'].unique())
print("Race: ", dfs['race'].unique())
print("Sex: ", dfs['sex'].unique())
print("Native Country: ", dfs['native-country'].unique())

print("Earn: ", dfs['earn'].unique())

Workclasses:  [' State-gov' ' Self-emp-not-inc' ' Private' ' Federal-gov' ' Local-gov'
 ' ?' ' Self-emp-inc' ' Without-pay' ' Never-worked']
Fnlwgt:  [ 77516  83311 215646 ...  34066  84661 257302]
Education:  [' Bachelors' ' HS-grad' ' 11th' ' Masters' ' 9th' ' Some-college'
 ' Assoc-acdm' ' Assoc-voc' ' 7th-8th' ' Doctorate' ' Prof-school'
 ' 5th-6th' ' 10th' ' 1st-4th' ' Preschool' ' 12th']
Marital Status:  [' Never-married' ' Married-civ-spouse' ' Divorced'
 ' Married-spouse-absent' ' Separated' ' Married-AF-spouse' ' Widowed']
Occupation:  [' Adm-clerical' ' Exec-managerial' ' Handlers-cleaners' ' Prof-specialty'
 ' Other-service' ' Sales' ' Craft-repair' ' Transport-moving'
 ' Farming-fishing' ' Machine-op-inspct' ' Tech-support' ' ?'
 ' Protective-serv' ' Armed-Forces' ' Priv-house-serv']
Relationship:  [' Not-in-family' ' Husband' ' Wife' ' Own-child' ' Unmarried'
 ' Other-relative']
Race:  [' White' ' Black' ' Asian-Pac-Islander' ' Amer-Indian-Eskimo' ' Other']
Sex:  [' Male' 

In [28]:
### Preprocessing
from sklearn.preprocessing import LabelEncoder

nunique = dfs.nunique()
types = dfs.dtypes

categorical_columns = []
categorical_dims = {}

## Label Encoding
for col in dfs.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(col, dfs[col].nunique())
        l_enc = LabelEncoder()
        dfs[col] = dfs[col].fillna("Fill Na")
        dfs[col] = l_enc.fit_transform(dfs[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)


age 73
workclass 9
education 16
education-num 16
marital-status 7
occupation 15
relationship 6
race 5
sex 2
capital-gain 119
capital-loss 92
hours-per-week 94
native-country 42
earn 2


In [29]:
len(dfs['workclass'].unique())

9

In [30]:
# Fill Nan Values with mean
dfs_indexes = dfs.index
dfs.fillna(dfs.loc[dfs_indexes, col].mean(), inplace=True)

In [31]:
X = dfs[['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]
y = dfs['earn']

In [32]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,22,7,77516,9,12,4,1,1,4,1,25,0,39,39
1,33,6,83311,9,12,2,4,0,4,1,0,0,12,39
2,21,4,215646,11,8,0,6,1,4,1,0,0,39,39
3,36,4,234721,1,6,2,6,0,2,1,0,0,39,39
4,11,4,338409,9,12,2,10,5,2,0,0,0,39,5


In [33]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: earn, dtype: int64

In [34]:
# ## LABEL ENCODING
# # convert categorical features to variables 0, 1, 2 etc. 

# from sklearn import preprocessing
# # Reset target
# le_earn = preprocessing.LabelEncoder()
# le_earn.fit([' <=50K',' >50K'])   # reset earn to 0 and 1
# y[:,0] = le_earn.transform(y[:,0]) 


# # Transform work
# le_work = preprocessing.LabelEncoder()
# le_work.fit([' Self-emp-not-inc', ' Private', ' State-gov',' Federal-gov', ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay', ' Never-worked'])
# X[:,1] = le_work.transform(X[:,1])

# # Transform Education
# le_edu = preprocessing.LabelEncoder()
# le_edu.fit([' Bachelors', ' HS-grad',' 11th', ' Masters', ' 9th', ' Some-college', ' Assoc-acdm',' Assoc-voc',' 7th-8th',' Doctorate',' Prof-school', ' 5th-6th',' 10th',' 1st-4th',' Preschool',' 12th'])
# X[:,3] = le_work.transform(X[:,3])



In [35]:
from sklearn.model_selection import train_test_split

train_ratio = 0.8
validation_ratio = 0.1
test_ratio = 0.10

# Divide to 80/10/10
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio)) 

train_indices = X_train.index
valid_indices = X_val.index
test_indices = X_test.index

In [36]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)
X_test = np.asarray(X_test)
y_test = np.asarray(y_test)

In [56]:
# Feature Transformer class

class FeatureTransformer(nn.Module):
    def __init__(self, input_dim, output_dim):
      super(self).__init__()

      # A class implementing the feature transformer architecture as seen in the paper page 4 (https://arxiv.org/pdf/1908.07442.pdf)
      self.model = torch.nn.Sequential(
          # First fully connected layer
          nn.Linear(input_dim,output_dim*2),
          # BatchNorm Layer
          nn.BatchNorm2d(output_dim),  # BN Layer With Learnable Parameters
          # Gated linear unit (GLU) Layer
          nn.GLU()
      )

    def forward(self, input):
      output = self.model(input)
      return output
    
    # Shared Accross Decision step
    output = forward(inputs)
    output1 = output
    output_part_2 = self.forward(output)
    output2 = output1 + output_part_2
    output2 = output2 / np.sqrt(0.5)

    # Decision step dependent
    output_part_3 = self.forward(output2)
    output3 = output2 + output_part_3
    output3 = output3 / np.sqrt(0.5)

    output_part_4 = self.forward(output3)
    output4 = output3 + output_part_4
    output4 = output4 / np.sqrt(0.5)


ft = FeatureTransformer()
#print(ft)

NameError: ignored

In [45]:
# Attentive Transformer

class AttentiveTransformer(nn.Module):
    def __init__(self, input_dim, output_dim):
      super(self).__init__()

      # A class implementing the feature transformer architecture as seen in the paper page 4 (https://arxiv.org/pdf/1908.07442.pdf)
      
      # First fully connected layer
      self.fc = nn.Linear(input_dim, output_dim)
      # BatchNorm Layer
      self.bn = nn.BatchNorm2d(output_dim)  # BN Layer With Learnable Parameters  
      # Sparsemax
      

    def sparseMax(inputs):
      pass  

    def forward(self, input, prior):
      output = self.fc(input)
      output = self.bn(output)
      output = torch.mul(output, prior)
      output = sparseMax(output)

      prior = output

      return output, prior

at = AttentiveTransformer()
#print(at)

NameError: ignored

In [ ]:
class TabNet(nn.Module):
    def __init__(self,inp_dim,final_out_dim,n_d=64,n_a=64,
n_shared=2,n_ind=2,n_steps=5,relax=1.2,vbs=128):
        super().__init__()
        if n_shared>0:
            self.shared = nn.ModuleList()
            self.shared.append(nn.Linear(inp_dim,2*(n_d+n_a)))
            for x in range(n_shared-1):
                self.shared.append(nn.Linear(n_d+n_a,2*(n_d+n_a)))
        else:
            self.shared=None
        self.first_step = FeatureTransformer(inp_dim,n_d+n_a,self.shared,n_ind) 
        self.steps = nn.ModuleList()
        for x in range(n_steps-1):
            self.steps.append(DecisionStep(inp_dim,n_d,n_a,self.shared,n_ind,relax,vbs))
        self.fc = nn.Linear(n_d,final_out_dim)
        self.bn = nn.BatchNorm1d(inp_dim)
        self.n_d = n_d
    def forward(self,x):
        x = self.bn(x)
        x_a = self.first_step(x)[:,self.n_d:]
        sparse_loss = torch.zeros(1).to(x.device)
        out = torch.zeros(x.size(0),self.n_d).to(x.device)
        priors = torch.ones(x.shape).to(x.device)
        for step in self.steps:
            x_te,l = step(x,x_a,priors)
            out += F.relu(x_te[:,:self.n_d])
            x_a = x_te[:,self.n_d:]
            sparse_loss += l
        return self.fc(out),sparse_loss

In [ ]:
Class TabNetDecoder(torch.nn.Module):


## Feature transformer method
self.feature_transformer():

